<a href="https://colab.research.google.com/github/Tikuh5/2024_Geoinformatique_I/blob/main/surface_agricole_utile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas pandas pooch

import pooch



In [3]:
# URL du fichier ZIP
url = "https://files.swisscarto.ch/geoinf1/vd_dgav_surfaces_agricoles.zip"

# Définir le chemin d'enregistrement du fichier ZIP
filename = pooch.retrieve(url, known_hash=None)

# Décompresser l'archive
import zipfile
import os

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('/content/surfaces_agricoles')

import geopandas as gpd



SHA256 hash of downloaded file: 0d866067c647f235d92499e669b8dfb4dfea18da3cfd99434a7e6176a9fba81f
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


In [4]:
# Charger le Geopackage contenant la couche "agr_dgav_sau"
gpk_path = "/content/surfaces_agricoles/vd_dgav_surfaces_agricoles.gpkg"
gdf_agriculture = gpd.read_file(gpk_path, layer='agr_dgav_sau')

from google.colab import files

In [5]:
# Télécharger le fichier depuis votre ordinateur
uploaded = files.upload()

Saving vd_dgav_surfaces_agricoles.gpkg to vd_dgav_surfaces_agricoles.gpkg


In [6]:
# Lire le fichier Geopackage téléchargé
filename = list(uploaded.keys())[0]  # Récupérer le nom du fichier téléchargé
gdf_perimetre = gpd.read_file(filename)  # Charger le fichier dans GeoPandas

/usr/local/lib/python3.10/dist-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'vd_dgav_surfaces_agricoles.gpkg': 'agr_dgav_sau' (default), 'agr_dgav_prom_biodiv_reseau', 'agr_dgav_surface_estivage'. Specify layer parameter to avoid this warning.
  result = read_func(


In [7]:
# Intersection des polygones des surfaces agricoles avec le périmètre
gdf_agriculture_intersected = gpd.overlay(gdf_agriculture, gdf_perimetre, how='intersection')

import pandas as pd

<ipython-input-7-48a8b74a018a>:2: UserWarning: `keep_geom_type=True` in overlay resulted in 247638 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  gdf_agriculture_intersected = gpd.overlay(gdf_agriculture, gdf_perimetre, how='intersection')


In [8]:
# Lire le fichier CSV
csv_path = '/content/surfaces_agricoles/vd_dgav_sau_utilisation_2023.csv'
df_utilisation = pd.read_csv(csv_path)

In [10]:
print(gdf_agriculture_intersected.columns)

Index(['ident_1', 'ident_2', 'geometry'], dtype='object')


In [11]:
gdf_agriculture_intersected.head()

,ident_1,ident_2,geometry
0,vd_406924,vd_406924,"POLYGON ((2499159.678 1130281.525, 2499129.647..."
1,vd_406924,vd_403135,"POLYGON ((2499159.678 1130281.525, 2499129.647..."
2,vd_406924,vd_403134,"POLYGON ((2498924.238 1130530.417, 2498974.55 ..."
3,vd_49133,vd_49133,"POLYGON ((2499091.525 1130629.275, 2498967.026..."
4,vd_49133,vd_413528,"POLYGON ((2499174.994 1130660.812, 2499169.74 ..."


In [13]:
!ls /content/

sample_data  surfaces_agricoles  vd_dgav_surfaces_agricoles.gpkg


In [16]:
# Effectuer la jointure sur la colonne 'ident' pour ajouter toutes les informations du CSV
sau_data = gdf_agriculture_intersected.merge(df_utilisation, left_on='ident_1', right_on= 'IDENT', how='left')

In [17]:
# Calcul de la superficie en hectares (l'unité est en mètres carrés par défaut)
sau_data['surface_ha'] = sau_data.geometry.area / 10000

In [18]:
# Regrouper par utilisation et calculer la somme des surfaces
resume_surfaces = sau_data.groupby('UTILISATIO')['surface_ha'].sum().reset_index()


In [19]:
# Sauvegarder le résumé dans un fichier CSV
resume_surfaces.to_csv('/content/resume_surfaces_agricoles.csv', index=False)

In [20]:
# Sauvegarder le GeoDataFrame dans un Geopackage
del(sau_data['IDENT'])
sau_data.to_file('/content/surfaces_agricoles_utilisation.gpkg', layer='surfaces_agricoles_utilisation', driver="GPKG")

from google.colab import files

In [21]:
# Télécharger les fichiers générés
files.download('/content/resume_surfaces_agricoles.csv')
files.download('/content/surfaces_agricoles_utilisation.gpkg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>